In [1]:
#Import Libraries
import findspark
findspark.init()
import json
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.functions import col
from pyspark.sql import SQLContext

In [2]:
# Create spark session
spark = SparkSession.builder \
    .master("local") \
    .appName("JSON Flatten") \
    .getOrCreate()

In [3]:
# Reading json data from Click Stream Data from Hadoop directory into dataframe 
df = spark.read.json('C:\\Users\\Futurense\\OneDrive\\Desktop\\CryptoPrediction\\resultstream.json')

In [4]:
# Extracting columns from json Objects in dataframe and create new dataframe with the extracted cloumns 
df.printSchema()

root
 |-- value_str: string (nullable = true)



In [5]:
from pyspark.sql.types import StructType,StructField, StringType, DoubleType, LongType
schema = StructType([ 
    StructField("price_usd",DoubleType(),True), 
    StructField("price_btc",LongType(),True), 
    StructField("price_eth",DoubleType(),True), 
    StructField("volume_last_24_hours", DoubleType(), True),
    StructField("real_volume_last_24_hours",DoubleType(),True), 
    StructField("volume_last_24_hours_overstatement_multiple",DoubleType(),True), 
    StructField("volume_last_24_hours_overstatement_multiple",DoubleType(),True), 
    StructField("percent_change_btc_last_1_hour", LongType(), True),
    StructField("percent_change_eth_last_1_hour",DoubleType(),True), 
    StructField("percent_change_usd_last_24_hours",DoubleType(),True), 
    StructField("percent_change_btc_last_24_hours",LongType(),True), 
    StructField("percent_change_eth_last_24_hours", DoubleType(), True),
    StructField("ohlcv_last_1_hour", StructType([
             StructField("open", DoubleType(), True),
             StructField("high", DoubleType(), True),
             StructField("low", DoubleType(), True),
             StructField("close", DoubleType(), True),
             StructField("volume", DoubleType(), True),
             ])),
    StructField("ohlcv_last_24_hour", StructType([
             StructField("open", DoubleType(), True),
             StructField("high", DoubleType(), True),
             StructField("low", DoubleType(), True),
             StructField("close", DoubleType(), True),
             StructField("volume", DoubleType(), True),
             ])),
    StructField("last_trade_at", StringType(), True)
  ])

In [6]:
dfJSON = df.withColumn("jsonData",from_json(col("value_str"),schema)) \
                   .select("jsonData.*")
dfJSON.printSchema()

root
 |-- price_usd: double (nullable = true)
 |-- price_btc: long (nullable = true)
 |-- price_eth: double (nullable = true)
 |-- volume_last_24_hours: double (nullable = true)
 |-- real_volume_last_24_hours: double (nullable = true)
 |-- volume_last_24_hours_overstatement_multiple: double (nullable = true)
 |-- volume_last_24_hours_overstatement_multiple: double (nullable = true)
 |-- percent_change_btc_last_1_hour: long (nullable = true)
 |-- percent_change_eth_last_1_hour: double (nullable = true)
 |-- percent_change_usd_last_24_hours: double (nullable = true)
 |-- percent_change_btc_last_24_hours: long (nullable = true)
 |-- percent_change_eth_last_24_hours: double (nullable = true)
 |-- ohlcv_last_1_hour: struct (nullable = true)
 |    |-- open: double (nullable = true)
 |    |-- high: double (nullable = true)
 |    |-- low: double (nullable = true)
 |    |-- close: double (nullable = true)
 |    |-- volume: double (nullable = true)
 |-- ohlcv_last_24_hour: struct (nullable = tru

In [7]:
finaldf = dfJSON.select(col('price_usd'), col('price_btc'), col('volume_last_24_hours'),col('percent_change_btc_last_24_hours'), col('last_trade_at'),col('ohlcv_last_1_hour.*'))
finaldf.show()

+------------------+---------+--------------------+--------------------------------+--------------------+-------+-----------------+------------------+------------------+--------------------+
|         price_usd|price_btc|volume_last_24_hours|percent_change_btc_last_24_hours|       last_trade_at|   open|             high|               low|             close|              volume|
+------------------+---------+--------------------+--------------------------------+--------------------+-------+-----------------+------------------+------------------+--------------------+
| 36060.49674629775|        1|5.745416053576366E13|                               0|2022-01-25T09:01:...|36181.0|36562.81427108558|36051.334920174704| 36060.12637680139| 1.624595216217537E8|
| 36066.78458606818|        1|5.745416053576366E13|                               0|2022-01-25T09:01:...|36181.0|36562.81427108558|36051.334920174704|36065.904753391056|1.6279619180093744E8|
| 36090.80725416168|        1|5.7454160535763

In [8]:
finaldf.write.option("header",True).csv("C:\\Users\\Futurense\\OneDrive\\Desktop\\CryptoPrediction\\Flatresult")